


# Training Private GPT



In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#!pip install transformers

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
#!pip install accelerate
# !pip install auto-gptq
# !pip install --upgrade optimum

In [ ]:
mn = 'TheBloke/Llama-2-7b-Chat-GPTQ'
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)

In [6]:
import datasets

We are going to use Know_ sql dataset to train our fine tunned model
The dataset source can be found here: https://huggingface.co/datasets/knowrohit07/know_sql

In [7]:
ds = datasets.load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['answer', 'context', 'question'],
        num_rows: 78562
    })
})

In [9]:
tr = ds["train"]
tr[3]

{'answer': "SELECT Hosts FROM farm_competition WHERE Theme <> 'Aliens'",
 'context': 'CREATE TABLE farm_competition (Hosts VARCHAR, Theme VARCHAR)',
 'question': 'What are the hosts of competitions whose theme is not "Aliens"?'}